# Data Analysis
# FINM August Review
# Homework Solution 4

## Imports

In [201]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Data

In [202]:
data = pd.read_excel("../data/single_name_return_data.xlsx", sheet_name="total returns").set_index("Date")

data.head()

,AAPL,AMZN,BA,BAC,C,CVX,DUK,EOG,FB,GS,...,MSFT,NEE,PFE,SHV,SPY,TMO,UNH,UPS,V,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-07-31,0.045822,0.021677,-0.005249,-0.102690,-0.010215,0.038673,-0.020237,0.089705,-0.301929,0.052576,...,-0.036613,0.030373,0.045217,0.000000,0.011829,0.072433,-0.126667,-0.039995,0.044002,0.014959
2012-08-31,0.093877,0.064166,-0.028167,0.088556,0.095507,0.031739,-0.033419,0.104989,-0.168125,0.052375,...,0.052738,-0.042232,0.001679,0.000181,0.025053,0.030178,0.062831,-0.016486,-0.004667,0.011714
2012-09-30,0.002796,0.024369,-0.025210,0.106515,0.101313,0.039229,0.000154,0.034626,0.199336,0.075293,...,-0.034394,0.044867,0.041492,-0.000081,0.025351,0.028059,0.024575,-0.030348,0.047018,0.047537
2012-10-31,-0.107600,-0.084264,0.012069,0.055493,0.142726,-0.054221,0.013891,0.041252,-0.025392,0.076619,...,-0.040994,-0.003839,0.000805,-0.000073,-0.018199,0.037906,0.010648,0.023474,0.033363,-0.003062
2012-11-30,-0.012374,0.082271,0.060996,0.057940,-0.075174,-0.033007,-0.016332,0.009700,0.326386,-0.033504,...,-0.059609,-0.010515,0.015068,0.000091,0.005660,0.040780,-0.028750,0.006248,0.081419,-0.027182


## 1 Penalized Regression

### 1.1

**Regress Pfizer (PFE) on the single-name equities, SPY, and SHV using OLS:**

In [203]:
y = data['PFE']
X = data.drop(columns=['PFE'])
model_ols = LinearRegression().fit(X,y)

### (a)

 Report the estimated $\alpha$ and $\beta$'s:

In [204]:
betas_ols = pd.DataFrame(model_ols.coef_, index = X.columns, columns = ['Parameters']) 
const_ols = pd.DataFrame(model_ols.intercept_, index = ['Intercept'], columns = ['Parameters'])

params_ols = pd.concat([const_ols, betas_ols])
params_ols

,Parameters
Intercept,0.001435
AAPL,-0.090211
AMZN,0.085638
BA,-0.097980
BAC,0.063022
C,-0.013769
CVX,0.275373
DUK,-0.022501
EOG,-0.055048
FB,-0.058740


###  (b)

Report $R^{2}$:

In [205]:
print("R-squared: " + str(round(model_ols.score(X, y),3)))

R-squared: 0.587


### (c)

Which factors have the largest betas in explaining $r^{i}$?

In [206]:
betas_ols.abs().nlargest(5, 'Parameters')

,Parameters
SHV,5.954589
SPY,0.869191
JNJ,0.408252
MSFT,0.303362
CVX,0.275373


SHV has the largest absolute $\beta$. SPY, JNJ, MSFT, and CVX have the next largest $\beta$ values.

### (d)

Calculate $\beta^{j}\sigma^{j}$ for each regressor.

Which of these is largest in magnitude, and thus most influential in explaining $r^{i}$?

In [207]:
betas_ols['Beta*vol'] = betas_ols['Parameters'] * X.std()
betas_ols

,Parameters,Beta*vol
AAPL,-0.090211,-0.007262
AMZN,0.085638,0.006911
BA,-0.097980,-0.009843
BAC,0.063022,0.005161
C,-0.013769,-0.001236
CVX,0.275373,0.019337
DUK,-0.022501,-0.001038
EOG,-0.055048,-0.005808
FB,-0.058740,-0.006040
GS,-0.169621,-0.013066


In [208]:
betas_ols.abs().nlargest(5, 'Beta*vol')

,Parameters,Beta*vol
SPY,0.869191,0.032663
CVX,0.275373,0.019337
JNJ,0.408252,0.017627
MSFT,0.303362,0.017434
XOM,0.236130,0.016014


SPY's $\beta \cdot \sigma$ is largest in magnitude, and thus it is the most influential in explaining $r^{i}$.

### 1.2

**Estimade the model using Ridge regression with a penalty of 0.5:**

In [209]:
model_ridge = Ridge(alpha=0.5).fit(X,y)

### (a)

Report the estimated $\alpha$ and $\beta$'s:

In [210]:
betas_ridge = pd.DataFrame(model_ridge.coef_, index = X.columns, columns = ['Parameters']) 
const_ridge = pd.DataFrame(model_ridge.intercept_, index = ['Intercept'], columns = ['Parameters'])

params_ridge = pd.concat([const_ridge, betas_ridge])
params_ridge

,Parameters
Intercept,-0.001788
AAPL,-0.015984
AMZN,0.078043
BA,-0.033985
BAC,0.024545
C,0.020820
CVX,0.059144
DUK,0.055371
EOG,0.011660
FB,-0.003222


### (b)

Report $R^{2}$:

In [211]:
print("R-squared: " + str(round(model_ridge.score(X, y),3)))

R-squared: 0.443


### (c)

Based on $\beta^{j}\sigma^{j}$, which factor is most influential for $r^{i}$?

In [212]:
betas_ridge['Beta*vol'] = betas_ridge['Parameters'] * X.std()
round(betas_ridge,4)

,Parameters,Beta*vol
AAPL,-0.0160,-0.0013
AMZN,0.0780,0.0063
BA,-0.0340,-0.0034
BAC,0.0245,0.0020
C,0.0208,0.0019
CVX,0.0591,0.0042
DUK,0.0554,0.0026
EOG,0.0117,0.0012
FB,-0.0032,-0.0003
GS,-0.0091,-0.0007


In [213]:
betas_ridge.abs().nlargest(1, 'Beta*vol')

,Parameters,Beta*vol
AMZN,0.078043,0.006298


Amazon's $\beta \cdot \sigma$ is largest in magnitude, and thus it is the most influential in explaining $r^{i}$.

### 1.3

**Estimate the model using Lasso with a penalty of 3e-4:**

In [214]:
model_lasso = Lasso(alpha=3e-4).fit(X,y)

### (a)

Report the estimated $\alpha$ and $\beta$'s:

In [215]:
betas_lasso = pd.DataFrame(model_lasso.coef_, index = X.columns, columns = ['Parameters']) 
const_lasso = pd.DataFrame(model_lasso.intercept_, index = ['Intercept'], columns = ['Parameters'])

params_lasso = pd.concat([const_lasso, betas_lasso])
params_lasso

,Parameters
Intercept,-0.002793
AAPL,-0.000000
AMZN,0.052032
BA,-0.000000
BAC,0.000000
C,0.000000
CVX,0.064272
DUK,0.000000
EOG,0.000000
FB,-0.000000


### (b)

Report $R^{2}$:

In [216]:
print("R-squared: " + str(round(model_lasso.score(X, y),3)))

R-squared: 0.455


### (c)

Based on $\beta^{j}\sigma^{j}$, which factor is most influential for $r^{i}$?

In [217]:
betas_lasso['Beta*vol'] = betas_lasso['Parameters'] * X.std()
round(betas_lasso,4)

,Parameters,Beta*vol
AAPL,-0.0000,-0.0000
AMZN,0.0520,0.0042
BA,-0.0000,-0.0000
BAC,0.0000,0.0000
C,0.0000,0.0000
CVX,0.0643,0.0045
DUK,0.0000,0.0000
EOG,0.0000,0.0000
FB,-0.0000,-0.0000
GS,0.0000,0.0000


In [231]:
betas_lasso.abs().nlargest(5, 'Beta*vol')

,Parameters,Beta*vol
JNJ,0.310078,0.013388
TMO,0.162022,0.009070
UNH,0.126808,0.007135
CVX,0.064272,0.004513
AMZN,0.052032,0.004199


Johnson & Johnson's $\beta \cdot \sigma$ is largest in magnitude, and thus it is the most influential in explaining $r^{i}$.

### (d)

How many regressors have a non-zero $\beta$ estimates?

In [219]:
print('Non-zero Beta Estimates: ' + str((betas_lasso['Parameters']!=0).sum()))

Non-zero Beta Estimates: 7


### (e)

How high do you need to set the penalty to get only 3 non-zero $\beta$ estimates?

In [232]:
### Set initial penalty
penalty = 3e-4

### Loop through penalty values until we find one that gets 3 non-zero beta estimates
while (Lasso(alpha=penalty).fit(X,y).coef_ != 0).sum() != 3:
    penalty += 1e-6
    
print('Penalty to have 3 non-zero terms: ' + str(round(penalty,8)))

Penalty to have 3 non-zero terms: 0.001293


### 1.4

### (a)

Create a table of estimated the betas and intercept across our three methods, (OLS, Ridge,
LASSO.) 
- Are they all nonzero? 
- Are there positive and negative values? 
- Do they range widely in magnitude?

In [221]:
params = pd.DataFrame(index = params_ols.index, columns = ['OLS','Ridge','Lasso'])
params['OLS'] = params_ols
params['Ridge'] = params_ridge
params['Lasso'] = params_lasso

params

,OLS,Ridge,Lasso
Intercept,0.001435,-0.001788,-0.002793
AAPL,-0.090211,-0.015984,-0.000000
AMZN,0.085638,0.078043,0.052032
BA,-0.097980,-0.033985,-0.000000
BAC,0.063022,0.024545,0.000000
C,-0.013769,0.020820,0.000000
CVX,0.275373,0.059144,0.064272
DUK,-0.022501,0.055371,0.000000
EOG,-0.055048,0.011660,0.000000
FB,-0.058740,-0.003222,-0.000000


In [222]:
for model in params.columns:
    print(model + ' non-zero estimates: ' + str((params[model] != 0).sum()))

OLS non-zero estimates: 24
Ridge non-zero estimates: 24
Lasso non-zero estimates: 8


- They are not all non-zero
- There are both positive and negative values
- Typically they do not vary enormously in magnitude. In OLS we see a very negative estimate of $\beta$ for SHV, most likely due to scaling of SHV. In Ridge and LASSO SHV is not given much weighting, if any at all. 

### (b)

Which model has the largest $R^{2}$? Is this a surprise?

In [223]:
r2 = pd.DataFrame(index = ['OLS', 'Ridge', 'Lasso'], columns = ['$R^{2}$'])

r2['$R^{2}$'] = [model_ols.score(X, y), model_ridge.score(X, y), model_lasso.score(X, y)]

round(r2, 4)

,$R^{2}$
OLS,0.5873
Ridge,0.4426
Lasso,0.4549


OLS has the largest $R^{2}$ which is not surprising at all as we are looking at in sample fit. OLS will fit the regressors to the regressand as well as it can in sample, while Ridge and LASSO will penalize when we weigh factors more heavily. As a result, OLS will fit better in sample, but it may be overfitting. 

## 2 Tuning and OOS

### 2.1

First, let’s consider a simple split of the sample. Re-do your estimations, (from problem 1.3,)
based on data through 2018 to fit the model, and then test the model on 2019-2021. (That is,
treat 2019-2021 as the OOS period.) 
- What is the $R^{2}$ in these out-of-sample fits?
- Which method does better out-of-sample?

In [224]:
X_train = X[:"2018"].copy()
y_train = y[:"2018"].copy()
X_test = X["2019":].copy()
y_test = y["2019":].copy()

In [225]:
ols_2018 = LinearRegression().fit(X_train,y_train)
ridge_2018 = Ridge(alpha=0.5).fit(X_train,y_train)
lasso_2018 = Lasso(alpha=3e-4).fit(X_train,y_train)

OOS_r2 = pd.DataFrame(index = ['OLS', 'Ridge', 'Lasso'], columns = ['$R^{2}$'])

OOS_r2['$R^{2}$'] = [ols_2018.score(X_test,y_test), ridge_2018.score(X_test,y_test), lasso_2018.score(X_test,y_test)]

round(OOS_r2, 4)

,$R^{2}$
OLS,0.3128
Ridge,0.2757
Lasso,0.3121


While they perform similarly, OLS and LASSO perform slightly better than Ridge, with OLS performing the best (barely).

### 2.2

We parameterized the penalties for Ridge and LASSO with ad-hoc numbers above. Instead, try
doing cross-validation. Make the entire sample, (through 2021,) available for the CV estimation.
Use the default number of folds.

In [226]:
model_ridgeCV = RidgeCV().fit(X,y)
ridgeCV_optimal = model_ridgeCV.alpha_

print('Ridge CV optimal parameter: ' + str(round(ridgeCV_optimal, 4)))
print('Ridge CV OOS R-squared: ' +str(round(model_ridgeCV.score(X_test, y_test), 4)))

Ridge CV optimal parameter: 0.1
Ridge CV OOS R-squared: 0.5749


In [227]:
model_lassoCV = LassoCV().fit(X,y)
lassoCV_optimal = model_lassoCV.alpha_

print('Lasso CV optimal parameter: ' + str(round(lassoCV_optimal, 4)))
print('Lasso CV OOS R-squared: ' +str(round(model_lassoCV.score(X_test, y_test), 4)))

Lasso CV optimal parameter: 0.0002
Lasso CV OOS R-squared: 0.5152


### 2.3

Let’s try back-testing the LASSO model, using the $\lambda$ value obtained from your CV in the
previous problem.
- Fit the model through June 30, 2017. Evaluate the model fit for the OOS period, July 31,
2017. Save the sample residual, the actual y value minus this OOS fit value.
- Repeat this same procedure, where in each iteration you fit the model through one month further, and you test the next step as the OOS value.
- With all of the saved OOS values, calculate the OOS $R^{2}$. For the baseline, use the expanding mean of the target, Pfizer.

In [228]:
### Find starting index 
starting_idx = len(X[:"2017-06-30"])

### Initialize the error lists 
err_x = np.array([])
err_null = np.array([])

for i in range(starting_idx, len(X) - 1):
    ### Data up to t
    curr_X = X.iloc[:i]
    curr_y = y.iloc[:i]
    
    ### Fit the model on data through t-1
    lasso_mod = Lasso(alpha=lassoCV_optimal).fit(curr_X, curr_y)

    ### Reshape the matrix so we can input it to predict
    predict_y = lasso_mod.predict(X.iloc[i].values.reshape(1, -1))[0]
    
    ### Actual OOS value
    actual_y = y.iloc[i]
    
    ### The expanding mean is our baseline for OOS r2, curr_y has data from time 1 to time t-1
    expanding_mean = curr_y.mean()
    
    ### Append the errors, which we will use to calculate OOS r2
    err_x = np.append(err_x, actual_y - predict_y)
    err_null = np.append(err_null, actual_y - expanding_mean)

In [229]:
r_sqr_oos = 1 - np.square(err_x).sum() / np.square(err_null).sum()
print('OOS r-squared: ' + str(round(r_sqr_oos, 4)))

OOS r-squared: 0.3153


This model does have look-ahead bias as we optimized the $\lambda$ parameter using the entire data set. We are using this value from data through 2021 when we do the "backtest".